Librarys to use

In [ ]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import folium
import gtfs_kit as gk
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium

General configurations

In [ ]:
# Ensure the OSMnx settings are optimized for your needs
ox.settings.use_cache = True
ox.settings.log_console = True


Functions

In [ ]:
def filter_bus_stops(gtfs_stops_file, center_point, radius=3000):
    """
    Load GTFS stops and filter those within a radius of the starting point.
    """
    # Load bus stops from GTFS stops.txt
    stops = pd.read_csv(gtfs_stops_file)
    stops_gdf = gpd.GeoDataFrame(
        stops,
        geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
        crs="EPSG:4326",
    )
    
    # Create a GeoDataFrame for the center point
    center_gdf = gpd.GeoDataFrame(
        {"geometry": [Point(center_point[1], center_point[0])]},
        crs="EPSG:4326",
    )
    
    # Buffer around the center point (in meters)
    center_buffer = center_gdf.to_crs(epsg=3812).buffer(radius).to_crs(epsg=4326)#4326
    
    # Filter stops within the buffer
    filtered_stops = stops_gdf[stops_gdf.geometry.within(center_buffer.iloc[0])]
    
    return filtered_stops

In [ ]:
# 2. Calculate shortest paths to bus stops
def calculate_shortest_paths_to_stops(graph, start_point, bus_stops):
    """
    Calculate shortest paths from the start point to each bus stop.
    """
    start_node = ox.distance.nearest_nodes(graph, start_point[1], start_point[0])
    paths = []

    for _, stop in bus_stops.iterrows():
        stop_coords = (stop.geometry.y, stop.geometry.x)
        stop_node = ox.distance.nearest_nodes(graph, stop_coords[1], stop_coords[0])
        try:
            shortest_path = nx.shortest_path(graph, source=start_node, target=stop_node, weight="length")
            path_length = sum(nx.get_edge_attributes(graph, "length").get((shortest_path[i], shortest_path[i+1], 0), 0) for i in range(len(shortest_path) - 1))
            paths.append({"stop_id": stop.stop_id, "path": shortest_path, "length": path_length,"stop_point":stop_node})
            #print(f"The length of the shortest path is {path_length:.2f} meters.")
        except nx.NetworkXNoPath:
            # Skip if there's no path
            continue
    
    return paths

In [ ]:
# 3. Plot the star graph
def plot_star_graph(graph, start_point, paths, bus_stops):
    """
    Plot the graph with shortest paths to bus stops.
    """
    
    # Initialize a folium map centered at the start point
    m = folium.Map(location=start_point, zoom_start=14)
    
    # Add the graph edges to the map
    for u, v, data in graph.edges(data=True):
        if "geometry" in data:
            coords = [(lat, lon) for lon, lat in data["geometry"].coords]
        else:
            coords = [(graph.nodes[u]["y"], graph.nodes[u]["x"]), (graph.nodes[v]["y"], graph.nodes[v]["x"])]
        folium.PolyLine(coords, color="gray", weight=1).add_to(m)
    
    # Add the start point marker
    folium.Marker(location=start_point, popup="Start", icon=folium.Icon(color="green")).add_to(m)
    
    # Add bus stops and paths
    for path_info in paths:
        stop_id = path_info["stop_id"]
        path = path_info["path"]
        stop = bus_stops[bus_stops.stop_id == stop_id].iloc[0]
        
        # Add path to map
        path_coords = [(graph.nodes[node]["y"], graph.nodes[node]["x"]) for node in path]
        folium.PolyLine(path_coords, color="red", weight=2).add_to(m)
        
        # Add bus stop marker
        stop_coords = (stop.geometry.y, stop.geometry.x)
        folium.Marker(location=stop_coords, popup=f"Stop ID: {stop_id}", icon=folium.Icon(color="blue")).add_to(m)
    
    # Return the map
    return m

In [ ]:
def create_gdf_with_paths(graph, start_point, paths, bus_stops):
    """
    Create a GeoDataFrame with the start point, bus stops, and paths.
    """
    # Start point as a geometry
    start_point_geom = Point(start_point[1], start_point[0])
    
    # Create a list of all nodes (start point + bus stops)
    all_nodes = [start_point_geom]
    all_paths = []

    for path_info in paths:
        stop_id = path_info["stop_id"]
        path = path_info["path"]
        stop_point = path_info["stop_point"]  # Correctly access stop_point

        # Create the path geometries (connecting the nodes)
        path_coords = [(graph.nodes[node]["x"], graph.nodes[node]["y"]) for node in path]
        for coords in path_coords:
            all_nodes.append(Point(coords))

        # Add the stop point to the nodes list
        all_nodes.append(stop_point)
    
    # Create a GeoDataFrame with the 'geometry' column
    gdf_nodes = gpd.GeoDataFrame(
        {'geometry': all_nodes}
    )
    
    # Set the CRS to 'EPSG:4326' after geometry is assigned
    gdf_nodes.set_crs("EPSG:4326", allow_override=True, inplace=True)
    
    # Add the type of node (start, path, stop)
    gdf_nodes['type'] = ['start'] + ['path'] * (len(all_nodes) - 2) + ['stop']
    
    return gdf_nodes


Main process

In [ ]:
# Def/home/lubuntu/.local/share/jupyter/runtimeine the starting location
location_start = "Mons, Belgium"
radius = 5000  # For graph creation
bus_stop_radius = 3000  # For filtering stops
    
# Path to GTFS stops.txt file
gtfs_stops_file = '/home/lubuntu/Téléchargements/GTFS/stops.txt' # Replace with your GTFS file path
    
# Get the center point
start_point = ox.geocode(location_start)
    
# Create the graph
graph = ox.graph_from_point(start_point, dist=radius, network_type="walk")
    
# Filter bus stops within the specified radius
filtered_stops = filter_bus_stops(gtfs_stops_file, start_point, radius=bus_stop_radius)
    
# Calculate shortest paths to bus stops
paths = calculate_shortest_paths_to_stops(graph, start_point, filtered_stops)
    
# Plot the star graph
star_map = plot_star_graph(graph, start_point, paths, filtered_stops)
#star_map.save("star_graph_map.html")
star_map